In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import json
import pickle

In [2]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# path = '/content/drive/My Drive/for_train.txt'
# text = open(path, 'r').read().lower()
# print('text length', len(text))

In [4]:
df = pd.read_csv('../data/for_train.csv')
df.head()

,char_line
0,"jerry: you know, why we're here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about ""we should go out""? this is what they're talking about...this whole thing, we're all out now, no one is home. not one person here is home, we're all out! there are people tryin' to find us, they don't know where we are. (imitates one of these people ""tryin' to find us""; pretends his hand is a phone) ""did you ring?, i can't find him."" (imitates other person on phone) ""where did he go?"" (the first person again) ""he didn't tell me where he was going"". he must have gone out. you wanna go out: you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...there you're staring around, whatta you do? you go: ""we gotta be getting back"". once you're out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, it's my feeling, you've gotta go. (pete's luncheonette. jerry and george are sitting at a table.)"
1,"jerry: seems to me, that button is in the worst possible spot. (talking about george's shirt) the second button literally makes or breaks the shirt, look at it: it's too high! it's in no-man's-land, you look like you live with your mother."
2,george: are you through? (kind of irritated)
3,"jerry: you do of course try on, when you buy?"
4,"george: yes, it was purple, i liked it, i don't actually recall considering the buttons."


In [5]:
df = df['char_line'].map(lambda x: x+'\n')

In [6]:
df.head()

0    jerry: you know, why we're here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about "we should go out"? this is what they're talking about...this whole thing, we're all out now, no one is home. not one person here is home, we're all out! there are people tryin' to find us, they don't know where we are. (imitates one of these people "tryin' to find us"; pretends his hand is a phone) "did you ring?, i can't find him." (imitates other person on phone) "where did he go?" (the first person again) "he didn't tell me where he was going". he must have gone out. you wanna go out: you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...there you're staring around, whatta you do? you go: "we gotta be getting back". once you're out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go ou

In [7]:
def seperate_punc(text):
    
    punc = ['...', '.', '[', ']', '(', ')', ';', ':', "'", '/', '"', ',', '?', '*', '!', '-', '$', '%', '&', '\n']
    for i in punc:
        text = text.replace(i, ' ' + i + ' ')
    text = text.replace('\n', '<NEWLINE>')
    return text

df = df.map(seperate_punc)

In [8]:
df[:5]

0    jerry :  you know ,  why we ' re here ?  to be out ,  this is out  .  .  .  and out is one of the single most enjoyable experiences of life .  people  .  .  .  did you ever hear people talking about  " we should go out "  ?  this is what they ' re talking about  .  .  .  this whole thing ,  we ' re all out now ,  no one is home .  not one person here is home ,  we ' re all out !  there are people tryin '  to find us ,  they don ' t know where we are .   ( imitates one of these people  " tryin '  to find us "  ;  pretends his hand is a phone )   " did you ring ?  ,  i can ' t find him .  "   ( imitates other person on phone )   " where did he go ?  "   ( the first person again )   " he didn ' t tell me where he was going "  .  he must have gone out .  you wanna go out :  you get ready ,  you pick out the clothes ,  right ?  you take the shower ,  you get all ready ,  get the cash ,  get your friends ,  the car ,  the spot ,  the reservation  .  .  .  there you ' re staring around ,

In [9]:
corpus = []
for text in df:
    corpus.append(text)
corpus

['jerry :  you know ,  why we \' re here ?  to be out ,  this is out  .  .  .  and out is one of the single most enjoyable experiences of life .  people  .  .  .  did you ever hear people talking about  " we should go out "  ?  this is what they \' re talking about  .  .  .  this whole thing ,  we \' re all out now ,  no one is home .  not one person here is home ,  we \' re all out !  there are people tryin \'  to find us ,  they don \' t know where we are .   ( imitates one of these people  " tryin \'  to find us "  ;  pretends his hand is a phone )   " did you ring ?  ,  i can \' t find him .  "   ( imitates other person on phone )   " where did he go ?  "   ( the first person again )   " he didn \' t tell me where he was going "  .  he must have gone out .  you wanna go out :  you get ready ,  you pick out the clothes ,  right ?  you take the shower ,  you get all ready ,  get the cash ,  get your friends ,  the car ,  the spot ,  the reservation  .  .  .  there you \' re staring a

In [10]:
tokenizer = Tokenizer(filters='', num_words=10000, char_level=False)

In [11]:
def preprocessing(text):
    tokenizer.fit_on_texts(text)
    token_list = tokenizer.texts_to_sequences(text)
    total_words = len(tokenizer.word_index) + 1
    
    input_sequences = []
    for i in range(len(token_list)):
        input_sequences += token_list[i]
    return input_sequences, total_words

inp_sequences, total_words = preprocessing(corpus)

In [12]:
def generator(input_sequences, max_len, total_words, batch_size=128):

    while True:
        index = np.random.randint(0,len(input_sequences) - max_len - batch_size - 1)

        X = np.zeros((batch_size, max_len))
        y = []
        for i, num in enumerate(range(index, index + batch_size)):
            X[i] = input_sequences[num: num + max_len]
            y.append(input_sequences[num + max_len])
        y = ku.to_categorical(y, num_classes=total_words)
        
        yield X, y

In [13]:
max_len = 60
train = generator(inp_sequences, max_len, total_words)

In [14]:
def create_model(max_len, total_words):
    input_len = max_len
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, output_dim=100, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
#     model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
    
    return model

model = create_model(max_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 100)           1872500   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 18725)             2415525   
Total params: 4,405,273
Trainable params: 4,405,273
Non-trainable params: 0
_________________________________________________________________


In [15]:
checkpoint = ModelCheckpoint('weights.hdf5', monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [checkpoint, reduce_lr]

In [16]:
model.fit(train, 
          steps_per_epoch=2000, 
          epochs=20, 
          verbose=1, 
          callbacks=callbacks)

Epoch 1/20
2000/2000 [==============================] - ETA: 0s - loss: 4.4233
Epoch 00001: loss improved from inf to 4.42335, saving model to weights.hdf5
2000/2000 [==============================] - 392s 196ms/step - loss: 4.4233
Epoch 2/20
2000/2000 [==============================] - ETA: 0s - loss: 4.6470
Epoch 00002: loss did not improve from 4.42335
2000/2000 [==============================] - 391s 196ms/step - loss: 4.6470
Epoch 3/20
2000/2000 [==============================] - ETA: 0s - loss: 4.5552
Epoch 00003: loss did not improve from 4.42335
2000/2000 [==============================] - 397s 199ms/step - loss: 4.5552
Epoch 4/20
2000/2000 [==============================] - ETA: 0s - loss: 4.5791
Epoch 00004: loss did not improve from 4.42335
2000/2000 [==============================] - 393s 196ms/step - loss: 4.5791
Epoch 5/20
2000/2000 [==============================] - ETA: 0s - loss: 4.5827
Epoch 00005: loss did not improve from 4.42335
2000/2000 [=========================

In [2]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)   # fine tune preds, included temperature

In [3]:
def uppercase_char(generated):
    formatted = ''
    lines = generated.split('\n')
    for line in lines:
        char_line = line.split(':')
        if len(char_line) == 2:
            formatted_line = char_line[0].upper()+': '+char_line[1].strip().capitalize()+'\n'
            formatted_line = char_line[0].upper()+': '+char_line[1].strip().capitalize()+'\n'
        else:
            formatted_line = char_line[0].capitalize()+'\n'
        formatted += formatted_line
    return formatted

In [4]:
def generate_text(seed_text, next_words, model, max_len, temperature):
    
    generated = ''
    if seed_text:
        generated += seed_text.lower() + ' :'
    else:
        characters = ['JERRY', 'GEORGE', 'ELAINE', 'KRAMER', 'NEWMAN', 'MORTY', 'HELEN',
       'FRANK', 'SUSAN', 'ESTELLE', 'PETERMAN', 'WOMAN', 'PUDDY', 'MAN',
       'JACK', 'UNCLE LEO', 'MICKEY', 'STEINBRENNER', 'DOCTOR', 'CLERK']
        seed_text = np.random.choice(characters)
        generated += seed_text.lower() + ' :'
    
    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([generated])[0]
        token_list = pad_sequences([token_list], maxlen=max_len)
        predicted = model.predict(token_list, verbose=0)[0]

        next_index = sample(predicted, temperature)
        next_word = tokenizer.index_word[next_index]

        generated += " " + next_word
        
    # format the generated texts
    generated = generated.replace(' <newline> ', '\n')
    generated = generated.replace(' <newline>', '\n')
    generated = generated.replace('<newline> ', '\n')

    punc1 = ['.', ':', '!', ';', ')', ']', '?', ',', '%']
    for i in punc1:
        generated = generated.replace(' '+i, i)
    punc2 = ['[', '(', '$']    
    for i in punc2:
        generated = generated.replace(i+' ', i)
    punc3 = ["'", '-']    
    for i in punc3:
        generated = generated.replace(' '+i+' ', i)
    
    generated = uppercase_char(generated)
    
    return generated

In [112]:
for i in range(10):
    print(generate_text('jerry', 400, model, 60, 1))
    print('======================')

JERRY: (up) i told you.
ELAINE: ?
JERRY: What am i supposed to say?
ELAINE: Yeah, i'm there.
JERRY: What's going to ken cannot it.
JERRY: Oh, you've had some one of your name as i would do your friends. i'm really bad anyway.
ELAINE: Yeah.
HE: (playing) you should be a you in fact! i gotta go to'em!
JERRY: You don't know what you're the going to. you got your are
ELAINE: Well, i'm not an old.
GEORGE: (out) well, you doin'in no big on a driver thing happened.
ELAINE: Oh., you'll be.
ELAINE: Kramer and susan are all the movie.
MORTY: I'm going to you've i, to take a man, not the it.
JERRY: I want you to that whole head and not the old.
KRAMER: Well no, can i take it " (beside kramer's off) but i can't find me and pretty elaine.
HELEN: You're not pretty bad thing. (points) (jerry)
JERRY: He's just just a isn't not good and i are. a real
GEORGE: What's he.
ELAINE: What are you talking about him?
GEORGE: I can't believe you.
GEORGE: I see you?
GEORGE: No, he knows what can't he was the in t

In [10]:
    print(generate_text('obama', 400, model, 60, 1))

OBAMA: I thought he did she tries to be sorry. (signed you're are just)
KRAMER: Would you hear that?
SUSAN: They say, it's going back in your i to the coffee shop?
JERRY: What?
MORTY: So if you said, i haven't like your name.
JERRY: You're not in real they were good.
JERRY: If so that's there they were just very important.
KRAMER: And what's a time.
Following stupid. how much was for now? i just like that so of course didn't you give me get our three "
GEORGE: What? happy, don't of to'too.
JERRY: It. i can't..
ELAINE: It-he doesn't have one.
JERRY: Well, i was thinking to mrs.
GEORGE: What was that friends does well if you've still have i can't believe they um, i'll get it in the thing.
GEORGE: Telling ya. they go.
KRAMER: And it! it's car.
KRAMER: Is the food?
ELAINE: It's good.
JERRY: Fine..
KNOW: I don't want to seen that like you for the guy. george and he that one '. i'll be right.
MORTY: Like never been, but i'm stupid bad for (to jerry)
ELAINE: I mean we, you're going to talk ab

In [121]:
for i in range(10):
    print(generate_text('george', 400, model, 60, 1))
    print('======================')

GEORGE: Because i don't even actually like gonna have some new from the that shirt. i mean i was married.
KRAMER: And uh i'm saying...
Go at, it.
KRAMER: You've been going to get at the new.
NEWMAN: You?
ELAINE: Well i can't again.
JERRY: Yeah. pretty nice car, but that she just d because i don't get it. you're just watching do!
KRAMER: You will give them when he's like an old-you.
JERRY: I not am.
ELAINE: I don't care for me.
ELAINE: (pointing and the) (think she says " good woman. maybe.
GEORGE: Because you're for that water.
ELAINE: He have to have!
KRAMER: You. they the just walking for you've been a her of to and like you)
CARRIE: Oh! well, they'll be never job.
ELAINE: Because i. you were
MORTY: Alright, could do it a funny. (in the apartment)
JERRY: Hey.
ELAINE: We'm mr. pitt.
ELAINE: No, no.
JERRY: What was that just?
ELAINE: (pleading) elaine both elaine. now you jerry that is just take up door, you had jane.
MORTY: It was gonna have it. (kramer enters) (elaine enters)
JERRY: 

In [122]:
for i in range(10):
    print(generate_text('elaine', 400, model, 60, 1))
    print('======================')

ELAINE: You should be supposed to first what's my parents.
You y'one of her your shirt. (scene ends) they're not gonna out of the door and the i didn't see the real
ELAINE: (you is)
HELEN: (she leaves)
GEORGE: 
KRAMER: Look at the " very nice time,'you some guy?
KRAMER: We've need a movie!
KRAMER: Yeah, but i'm gonna go.
ELAINE: You're some woman?
GEORGE: So you?
NEWMAN: I don't think you're than him.
JERRY: When you got it.
JERRY: I got it to be.
JERRY: I can't not we never remember that.
NOW: What?
HELEN: And you're going to get.
GEORGE: 
?
KRAMER: You know, you'll get.
GEORGE: Come out.
ELAINE: Now! jerry. they have you.
JERRY: She like he's are the bathroom.
JERRY: Are you not gonna go a. i don't understand you my hair.
JERRY: Well, but it's) i gotta tell you why about kramer, i. he the street to them?
JERRY: We catch a little new watch)
RIGHT: No, no.
JERRY: No, i don't come to make any very good day.
COAT: You.
KRAMER: Yeah. i gotta go now.
MRS! ROSS: You're meeting. very she
JER

In [123]:
for i in range(10):
    print(generate_text('kramer', 400, model, 60, 1))
    print('======================')

KRAMER: Because i don't care of and
ELAINE: When we were thinking of his touch with me?
GEORGE: I'm getting together, you did their * of those people of the the no guy.
HELEN: I'm getting?
GEORGE: I don't know..'s going back?
ELAINE: You're elaine?
JERRY: How did you go? how'd you get it?
ELAINE: She's bad. (walks out) oh, well i'm sorry, how are you gonna kill him? what's your clothes.
ELAINE: Well, this is, then who's.
KRAMER: The i guess they can't be very much of
ELAINE: I could be gonna tell us?
JERRY: For one to.
KRAMER: Elaine, you don't stop out
ELAINE: (is playing left)
HELEN: It's going up for the way that going to live?
NEWMAN: Here, i don't say anything. you go back in fact, they're supposed to live to be
ELAINE: I didn't understand
JERRY: It's the be having the because of your two.
KRAMER: He can't pants, come on's too nice!
Was the.
ELAINE: At a dinner.
ELAINE: I don't know how many is so much! i more man.
KRAMER: Yeah, it!
GEORGE: That's called they.
ELAINE: Yeah.
ELAINE

In [124]:
for i in range(10):
    print(generate_text('frank', 400, model, 60, 1))
    print('======================')

FRANK: He's food.
KRAMER: But is they using it.
KRAMER: I'm going to go the last thing.. this guy.
ELAINE: I am, because you?
JERRY: She's.
ELAINE: Yes!
JERRY: Yeah. so now you would have a fred. will i get up in a bad one o'clock and they're the very elaine.
JERRY: Nothing.
, but it's the give me. you put a few.
Uh, i have it to a nice life. well, i don't need an
MORTY: I don't know anything you know. she's just because it.
HELEN: You're both just believe, they don't know how he said it i was it.
KRAMER: Kramer and you what those are.
GEORGE: I'm an old.
ELAINE: You've got to his other truck!
JERRY: I'm not not your job huh, i'm gonna you got a couple of the.
KRAMER: .... some deal and i'm big guy.
HELEN: I'm going out so (kramer enters) the'me again's.
GEORGE: 
JERRY: I been in the.
ELAINE: I did them on the believe her thing. i'm saying.
ELAINE: Where'd you. you later.
ELAINE: But i told you'd do you two like nice.
ELAINE: Me. you don't be the paper.
KRAMER: Well an this with the el

In [6]:
for i in range(10):
    print(generate_text('jerry', 400, model, 60, 0.33))
    print('======================')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
JERRY: I'm not gonna do.
ELAINE: I'm not a big deal.
ELAINE: Well, i'm very nice to be
JERRY: I'm not a pretty good.
ELAINE: I'm not going to be
ELAINE: I'm not a great.
JERRY: I'm not a pretty good.
JERRY: I'm not a big deal.
JERRY: I'm not gonna be a nice.
JERRY: I'm gonna be a big deal.
GEORGE: I'm a pretty bad.
ELAINE: Yeah.
JERRY: I'm not gonna go to the first.
GEORGE: I

In [8]:
for i in range(10):
    print(generate_text('', 400, model, 60, 0.66))
    print('======================')

MAN: 
JERRY: I'm thinking.
GEORGE: (to jerry) you're going to be a to frank.
HELEN: You'll be the job.
KRAMER: I'm sure you're gonna be so much.
ELAINE: I remember some's just like it.
ELAINE: Yeah.
JERRY: I'm not gonna go to the one.
JERRY: I'm going to be good.
JERRY: No, it's this.
ELAINE: No.
JERRY: I don't even want to be or
JERRY: Well, i'm gonna get it.
KRAMER: 
ELAINE: I'm it.
ELAINE: Well you're
ELAINE: I'm not that.
ELAINE: You don't think that's like you.
JERRY: How do you do me?
ELAINE: Oh, i don't know it.
ELAINE: I've
KRAMER: No.
ELAINE: I'm not gonna be pretty pretty much.
GEORGE: Oh, i'm going.
GEORGE: I'm've got to see if they were going out here.
ELAINE: You can be pretty good.
JERRY: I never got the first thing.
JERRY: I don't want you out.
JERRY: You were trying to say it?
ELAINE: I'm elaine.
ELAINE: Well you're my name, but you're the
ELAINE: I'm bad.
ELAINE: I can't even seen him.
JERRY: I don't know.
GEORGE: You're just like that.
GEORGE: Oh. you'll be like a kin

### Save Model

In [125]:
# Save the model full and lightweight version
model.save('../assets/rnn')
model.save('../assets/rnn.h5')

# Save the model architecture
model_json = model.to_json()
with open("../assets/rnn_config.json", "w") as config:
    config.write(model_json)

# Save the tokenizer to json
import io

tokenizer_json = tokenizer.to_json()
with io.open("../assets/rnn_tokenizer.json", "w") as token:
    token.write(json.dumps(tokenizer_json, ensure_ascii=False))

print("Saved model to disk")

In [5]:
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.preprocessing import text
import json

model = load_model('../assets/lstm1/rnn')

with open('../assets/lstm1/rnn_tokenizer.json') as f:
    data = json.load(f)
    tokenizer = text.tokenizer_from_json(data)

#### Hdf5 version too superior will cause .h5 load error